<a href="https://colab.research.google.com/github/leonardgst/Time_Series_project/blob/main/Connection_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import ee
import pandas as pd

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mxsgkaAj-fPkJQDrf9yFze7ii_LhqPZMQ9ARp4isHNI&tc=L2siHW_NmvsoYnSmT182gGYgi8CJIPCzrbk0WYaZ088&cc=961Wnu_FtsdwClszpnWdZwmDIudIrkcTgXS0OS34Kx4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmaFCgXiSXLKBc1cH-eErDlDGATDihKbW1qosP4YS4LdVLQ2-bJeJE

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [33]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['longitude', 'latitude', 'datetime',  *list_of_bands]]

    return df

In [42]:
selected_var2 = ["snow_density", "skin_temperature", "total_precipitation", "surface_latent_heat_flux"]
selected_var = ["snow_density"]

In [43]:
image = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
dataset_region = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
region = dataset_region.filter(ee.Filter.Or(ee.Filter.eq('ADM1_NAME', 'Midi-Pyrenees'),ee.Filter.eq('ADM1_NAME', 'Languedoc-Rousillon')))
reg = region.geometry()
scale=11132
df_vide=[]
df_vide=pd.DataFrame(df_vide)
g=1
j=0
x=2
z=0
for i in range(12):
    startDate = ee.Date.fromYMD(2000+z,g,1)
    endDate = ee.Date.fromYMD(2000+j,x,1)
    g += 1
    x += 1
    if g>12:
        g=1
        z+=1
    if x>12:
        x=1
        j+=1
    image2=image.select(selected_var).filterDate(startDate, endDate)
    image2=image2.getRegion(reg,scale).getInfo()
    print(type(image2))
    occitanie_prec_df=ee_array_to_df(image2,['total_precipitation'])
    occitanie_prec_df['yyyy'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.year
    occitanie_prec_df['mm'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.month
    occitanie_prec_df['dd'] = pd.to_datetime(occitanie_prec_df['datetime']).dt.day
    occitanie_prec_df['date']=occitanie_prec_df['yyyy'].astype('str') + '-'+ occitanie_prec_df['mm'].astype('str')+ '-'+ occitanie_prec_df['dd'].astype('str')
    data=occitanie_prec_df.drop(['datetime', 'yyyy', 'mm', 'dd'], axis=1)
    #moyenne des données des heures par jour
    mean_data=data.groupby(['longitude', 'latitude','date'],as_index=False, sort=False).mean()
    #Construction d'un point geometry comme geopanda mais sans la compléxité du dernier
    mean_data['Point']='Point('+mean_data['longitude'].astype('str') + ' '+ mean_data['latitude'].astype('str')+ ')'
    data_test=mean_data.drop(['longitude', 'latitude'], axis=1)
    df_vide=pd.concat((df_vide,data_test),ignore_index=True, sort=True)

<class 'list'>


KeyError: ignored

In [35]:
df_vide.head()

,Point,date,total_precipitation
0,Point(2.6500121218469057 42.3501937208364),2000-1-1,0.000122
1,Point(2.6500121218469057 42.3501937208364),2000-1-2,0.000004
2,Point(2.6500121218469057 42.3501937208364),2000-1-3,0.000009
3,Point(2.6500121218469057 42.3501937208364),2000-1-4,0.000024
4,Point(2.6500121218469057 42.3501937208364),2000-1-5,0.000003
